In [1]:
!pip install accelerate \
    protobuf \
    sentencepiece \
    torch \
    git+https://github.com/huggingface/transformers \
    huggingface_hub

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-t7it11pt
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-t7it11pt
  Resolved https://github.com/huggingface/transformers to commit 33868a057c02f0368ba63bd1edb746be38fe3d90
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 37.1 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.47.0.dev0-py3-none-any.whl size=10057149 sha256=0b22919c3b3c8e20e1dbf16b058ffdae18870d4736cdbcb1b5bdf188d226aa09
  Stored in directory: /tmp/pip-ephem-wheel-cache-8yddtdsx/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully 

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer

In [3]:
from huggingface_hub import login
login()

In [4]:
import torch
print("CUDA available:", torch.cuda.is_available())

CUDA available: True


In [ ]:
if torch.cuda.is_available():
    model_id = "chandankh/Llama-2-7b-chat-finetune-test5"
    model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")
    model.cuda()
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.use_default_system_prompt = False

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [15]:
def chat_with_llama(prompt):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    input_ids = input_ids.to('cuda')
    output = model.generate(input_ids, max_length=256, num_beams=4, no_repeat_ngram_size=2)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    print(response)
    return response

# while True:
#     prompt = input("You: ")
#     response = chat_with_llama(prompt)
#     print("Llama:", response)

In [13]:
!pip install datasets bert_score sentence-transformers evaluate rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=cc2d3187af1d5a026bc75c33df5ba9e8d2d64c970d9fd23fd83795fc6e913b1b
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
import evaluate
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer
import torch
from bert_score import score as bert_score
from sentence_transformers import SentenceTransformer, util

# Load dataset
dataset = pd.read_csv("Science_Dataset.csv")  # Replace with your CSV file path
batch_size = 10  # Process 10 questions at a time

# Initialize evaluation metrics
bleu_metric = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")
sbert_model = SentenceTransformer('all-mpnet-base-v2')

In [26]:
# # Initialize lists to store results
# generated_answers = []
# bleu_scores = []
# rouge1_scores = []
# rouge2_scores = []
# rougeL_scores = []
# bert_scores = []
# sbert_cosine_scores = []

# # Process in batches
# for i in range(0, len(dataset), batch_size):
#     batch = dataset.iloc[i:i + batch_size]

#     for index, row in batch.iterrows():
#         question = row['question']
#         ground_truth = row['ground_truth']

#         if pd.isna(ground_truth):  # Check for NaN
#            print(f"Ground truth is NaN for question at index {index}, skipping...")
#            continue

#         if pd.isna(question):  # Check for NaN
#            print(f"Question is NaN for question at index {index}, skipping...")
#            continue

#         # Generate model answer
#         model_answer = chat_with_llama(question)
#         generated_answers.append(model_answer)

#         # Compute BLEU score
#         # Compute BLEU score
#         bleu = bleu_metric.compute(predictions=[model_answer], references=[ground_truth])
#         bleu_scores.append(bleu["bleu"])


#         # Compute ROUGE scores
#         # Compute ROUGE scores
#         rouge = rouge_metric.compute(predictions=[model_answer], references=[ground_truth])
#         rouge1_scores.append(rouge['rouge1'])
#         rouge2_scores.append(rouge['rouge2'])
#         rougeL_scores.append(rouge['rougeL'])

#         # Compute BERTScore
#         P, R, F1 = bert_score([model_answer], [ground_truth], lang="en")
#         bert_scores.append(F1.mean().item())

#         # Compute SBERT cosine similarity
#         embedding1 = sbert_model.encode(model_answer, convert_to_tensor=True)
#         embedding2 = sbert_model.encode(ground_truth, convert_to_tensor=True)
#         cosine_score = util.pytorch_cos_sim(embedding1, embedding2).item()
#         sbert_cosine_scores.append(cosine_score)

# # Add results to DataFrame
# dataset['model_answer'] = generated_answers
# dataset['bleu_score'] = bleu_scores
# dataset['rouge1_score'] = rouge1_scores
# dataset['rouge2_score'] = rouge2_scores
# dataset['rougeL_score'] = rougeL_scores
# dataset['bert_score'] = bert_scores
# dataset['sbert_cosine_score'] = sbert_cosine_scores

# # Save results to a new CSV file
# dataset.to_csv("evaluation_results.csv", index=False)
# print("Evaluation completed and results saved to evaluation_results.csv")

State Mendeleev s Periodic law.
In 1869, Dmitri Mendeléev, a Russian chemist, created the first version of the periodic table of elements. He arranged the elements in order of increasing atomic weight and grouped them into vertical columns (groups) based on their chemical properties. This was a major breakthrough in the field of chemistry, as it allowed scientists to see relationships between elements that had previously been overlooked. In this lesson, we will explore Menedeliev's periodic law and how it has impacted the way we understand and organize elements today.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Why did Mendeleev leave some gaps in the Periodic table?

In 1869, Dmitri Mendeléev, a Russian chemist, created the first version of the periodic table of elements. He arranged the elements in order of increasing atomic weight and placed them in rows (periods) based on their similar chemical properties. However, there were some elements that he could not include in his table because they had not yet been discovered at the time.
There are several reasons why Menedeliev left some spaces or "gaps"in his periodic tables:
1. Lack of information: Many elements were not well understood or known at that time, and therefore, their properties couldnot be predicted accurately. As a result, Mendedelieav had limited information to place them correctly on the table. 
2. Predictions: Mendendeev made some predictions about the properties of yet-to-be-discovered elements, which later proved to be correct. For example, he predicted the existence of an element with an atomic number of 43 (Europium), whic

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Define electropositivity.
Electropositivity, also known as electronegativity, is the ability of an atom or molecule to attract electrons in a covalent bond. It is a measure of the atom's tendency to acquire a positive charge during the formation of a bond with another atom. The term was introduced by the chemist Linus Pauling in 1932.


...

State any three reasons to justify that LPG is considered as an ideal fuel.
LPG (Liquefied Petroleum Gas) is widely used as a fuel source in various applications, including residential, commercial, and industrial sectors. Here are three key reasons why it's considered ideal:

1. Environmentally friendly: Unlike traditional fossil fuels like gasoline and diesel, Liqueficated Petroluem Gas (LP) emits significantly less pollutants and greenhouse gases during combustion. This makes it an attractive option for those looking to reduce their carbon footprint and contribute to a more sustainable future. According to the U.S Environmental Protection Agency (EPA), LNG produces 25% less carbon monoxide, 30% fewer nitrogen oxides, & 40-60 % fewer non-methane organic compounds compared to traditional gasolines. Additionally, it produces no soot or particulate matter, which are harmful to both human health and the environment. As a result, many governments around the world are promoting the use of c

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


What is watershed management system? List two benefits derived by the communities that participate’ in this system.
Watersheds are areas of land that drain water into a common body of water, such as a river, lake, or ocean. Watershed Management Systems (WMS) are designed to protect and manage these areas in a sustainable way, ensuring the long-term health of the ecosystem, water quality, and community well-being. Participating communities can benefit from WMS in several ways, including:
1. Improved Water Quality: By implementing best management practices (BMPs) and restoring degraded habitats, communities participating in W MS can improve the quality of their water resources. This can lead to safer and more reliable drinking water sources, as well as improved aquatic habitat for fish and other wildlife.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ValueError: Length of values (30) does not match length of index (31)

In [ ]:
# Initialize lists to store results
generated_answers = []
bleu_scores = []
rouge1_scores = []
rouge2_scores = []
rougeL_scores = []
bert_scores = []
sbert_cosine_scores = []

# Process in batches
for i in range(0, len(dataset), batch_size):
    batch = dataset.iloc[i:i + batch_size]

    for index, row in batch.iterrows():
        question = row['question']
        ground_truth = row['ground_truth']

        if pd.isna(ground_truth):  # Check for NaN
           print(f"Ground truth is NaN for question at index {index}, skipping...")
           continue

        if pd.isna(question):  # Check for NaN
           print(f"Question is NaN for question at index {index}, skipping...")
           continue

        # Generate model answer
        model_answer = chat_with_llama(question)
        generated_answers.append(model_answer)

        # Compute BLEU score
        bleu = bleu_metric.compute(predictions=[model_answer], references=[ground_truth])
        bleu_scores.append(bleu["bleu"])

        # Compute ROUGE scores
        rouge = rouge_metric.compute(predictions=[model_answer], references=[ground_truth])
        rouge1_scores.append(rouge['rouge1'])
        rouge2_scores.append(rouge['rouge2'])
        rougeL_scores.append(rouge['rougeL'])

        # Compute BERTScore
        P, R, F1 = bert_score([model_answer], [ground_truth], lang="en")
        bert_scores.append(F1.mean().item())

        # Compute SBERT cosine similarity
        embedding1 = sbert_model.encode(model_answer, convert_to_tensor=True)
        embedding2 = sbert_model.encode(ground_truth, convert_to_tensor=True)
        cosine_score = util.pytorch_cos_sim(embedding1, embedding2).item()
        sbert_cosine_scores.append(cosine_score)

# If you want to keep the original dataset length, fill missing values with NaN
dataset['model_answer'] = [None] * len(dataset)
dataset['bleu_score'] = [None] * len(dataset)
dataset['rouge1_score'] = [None] * len(dataset)
dataset['rouge2_score'] = [None] * len(dataset)
dataset['rougeL_score'] = [None] * len(dataset)
dataset['bert_score'] = [None] * len(dataset)
dataset['sbert_cosine_score'] = [None] * len(dataset)

# Update the valid entries in the original dataset
for idx, value in enumerate(generated_answers):
    dataset.loc[dataset.index[idx], 'model_answer'] = value
    dataset.loc[dataset.index[idx], 'bleu_score'] = bleu_scores[idx]
    dataset.loc[dataset.index[idx], 'rouge1_score'] = rouge1_scores[idx]
    dataset.loc[dataset.index[idx], 'rouge2_score'] = rouge2_scores[idx]
    dataset.loc[dataset.index[idx], 'rougeL_score'] = rougeL_scores[idx]
    dataset.loc[dataset.index[idx], 'bert_score'] = bert_scores[idx]
    dataset.loc[dataset.index[idx], 'sbert_cosine_score'] = sbert_cosine_scores[idx]

# Save results to a new CSV file
dataset.to_csv("evaluation_results_fine_tuned_model.csv", index=False)
print("Evaluation completed and results saved to evaluation_results_fine_tuned_model.csv")


What is watershed management system? List two benefits derived by the communities that participate’ in this system.
Watersheds are areas of land that drain water into a common body of water, such as a river, lake, or ocean. Watershed Management Systems (WMS) are designed to protect and conserve these areas by managing land use practices, water quality, and other factors that can impact the health of the water and surrounding ecosystems. Participating communities can benefit from WMS in several ways, including:
1. Improved Water Quality: By implementing best management practices (BMPs) and monitoring water runoff, communities participating in W MS can reduce the amount of pollutants entering local waterways, improving water clarity and overall water health. This can lead to safer and more enjoyable recreational activities, as well as better fish and wildlife habitats. 2. Enhanced Economic Opportunities: A healthy and well-managed water shed can also provide economic benefits to local co

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


OutOfMemoryError: CUDA out of memory. Tried to allocate 16.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 11.06 MiB is free. Process 18407 has 14.73 GiB memory in use. Of the allocated memory 14.52 GiB is allocated by PyTorch, and 93.98 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# Load the full evaluation results file
results_df = pd.read_csv('evaluation_results_fine_tuned_model.csv')

# Calculate average scores
average_scores = {
    'avg_bleu': results_df['bleu_score'].mean(),
    'avg_rouge1': results_df['rouge1_score'].mean(),
    'avg_rouge2': results_df['rouge2_score'].mean(),
    'avg_rougeL': results_df['rougeL_score'].mean(),
    'avg_bert_score': results_df['bert_score'].mean(),
    'avg_sbert_cosine': results_df['sbert_cosine_score'].mean()
}

# Display average scores
print("\nAverage Scores:")
for metric, score in average_scores.items():
    print(f"{metric}: {score:.4f}")



Average Scores:
avg_bleu: 0.0439
avg_rouge1: 0.2556
avg_rouge2: 0.0865
avg_rougeL: 0.1727
avg_bert_score: 0.8484
avg_sbert_cosine: 0.7410
